<a href="https://colab.research.google.com/github/ghespinosa/iMMAPColombia/blob/master/LimpiezaDeMatrizAsentamientos_(V_actual).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lisa Troconis y Gustavo Espinosa** &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp;   <img src= "https://i0.wp.com/immap.org/wp-content/uploads/2016/12/immap-logo-strap2.png?resize=300%2C45&ssl=1"> <br />
*Last updated: July 24th 2020*

# Limpieza de Matriz de Asentamientos

<br />

**GitHub Project Card:** https://github.com/ghespinosa/iMMAPColombia/projects/1#card-42450124


**OVERVIEW:** 
- Reads SuperMatrix dataframe and MasterDict dictionary
  - creates various sub-dictionaries to clean up the super matrix
- Encodes dummy variables for each of the reported elements (WASH only so far)
- Finds the most frequently reported response to polichotomous variables

**TO-DO:**
- Clean Matrix fully
    - uses if statements to turn old master matrix into new master matrix with easy to interpret values
    - clean Salud, SAN, Infraestructura/Multisector
    - check for mistakes manually

*See:* 
- https://github.com/ghespinosa/proyectoasentamiento
- https://docs.google.com/document/d/1ndzWfIIWBdPW4VuWG18oPgbHN-3HJlB4y4DSGqgr9WM/edit?usp=sharing

## Import Libraries and Data

In [ ]:
# Import my Google Drive to be able to access files
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import pandas as pd
import pickle

import numpy as np

import math as mt

import matplotlib
import matplotlib.pyplot as plt

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Load in the pickled Super Matrix that is stored in the Google Drive
SuperMatrix = pickle.load( open( '/content/drive/Shared drives/iMMAP Colombia/Equipo Datos/PREMISE/Asentamientos Nuevos/AnalisisDeAsentamientos (en Python)/SuperMatrix.p', "rb" ) )

In [ ]:
# Load in the pickled Master Dictionary that is stored in the Google Drive
MasterDict = pickle.load( open( '/content/drive/Shared drives/iMMAP Colombia/Equipo Datos/PREMISE/Asentamientos Nuevos/AnalisisDeAsentamientos (en Python)/MasterDict.p', "rb" ) )

In [ ]:
# Load in the cleaned up but un-merged validation and monitoring CSVs
validES = pickle.load( open( '/content/drive/Shared drives/iMMAP Colombia/Equipo Datos/PREMISE/Asentamientos Nuevos/AnalisisDeAsentamientos (en Python)/ValidES.p', "rb" ) )
monitES = pickle.load( open( '/content/drive/Shared drives/iMMAP Colombia/Equipo Datos/PREMISE/Asentamientos Nuevos/AnalisisDeAsentamientos (en Python)/MonitES.p', "rb" ) )

## Data Exploration and Dictionary Management

In [ ]:
# Print some of the SuperMatrix
SuperMatrix.head(3) #look into why location_name is missing (probably because it was indexed and I dropped it)

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,form_id,submission_id,locale,location_id,location_name,location_centroid,. Por favor tome una foto del asentamiento,. Por favor tome una foto del asentamiento desde un ángulo diferente,. ¿Existe un asentamiento en este punto?,. ¿Sabe desde hace cuanto comenzó el asentamiento?,. ¿Las calles están pavimentadas?,. Por favor tome una foto de una calle que muestre si está o no pavimentada,. Seleccione el aspecto de la comunidad que está reportando,. Díganos desde qué lugar esta reportando este elemento,. Por favor tome una foto del elemento que está reportando,. Por favor escriba en pocas palabras a qué se refiere con “otro” en la pregunta anterior.,. ¿El material de construcción de las viviendas parece ser improvisado?,. ¿Las casas tienen puertas?,. ¿De qué materiales están hechas las viviendas principalmente?,. Por favor tome una fotografía afuera de una vivienda o grupo de viviendas que nos permita entender mejor el tipo de material y cómo están construidas.,. ¿Sabe el nombre de este asentamiento?,. Por favor escriba el nombre de este asentamiento,. ¿Los residentes actuales del asentamiento son colombianos?,. Díganos desde qué lugar está reportando ete elemento,. Por favor tome una fotografía del elemento que está reportando,. Los establecimientos de venta de alimentos y abarrotes son en su mayoría:,. ¿Qué alimentos y abarrotes venden dentro del establecimiento?,. ¿Cuáles de los siguientes alimentos están disponibles dentro del establecimiento?,. La calidad de los alimentos ofertados es:,. Por favor tome una fotografía de un anaquel con algunos productos dentro de la tienda.,". ¿Cuántas cultivos, unidades agrícolas o pecuarias hay en este asentamiento?",. ¿Qué tipos de productos observa en los cultivos o en las unidades productivas identificadas?,. En términos generales ¿cómo son las condiciones de las viviendas?,. ¿Qué tipos de instalaciones públicas sanitarias existen para los habitantes del asentamiento?,. ¿ Cuántas construcciones (viviendas) nuevas percibe que hay?,". ¿El Asentamiento cuenta con algún espacio comunitario para la transformación adecuada de los productos alimenticios para consumo, como cocinas comunitarias o comedores?",. ¿Cuál es la capacidad de dichas cocinas comunitarias o comedores?,. ¿Cuánto tiempo requiere para acudir a la institución o centro de salud más cercana al asentamiento?,. ¿Cómo calificaría la calidad de la señal de celular en la zona?,. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?,. ¿Cómo describiría el área en la que se encuentra el asentamiento?,. ¿Dentro del asentamiento se percibe gente adulta mayor?,. ¿Qué medidas observa se están implementando para prevenir o mitigar el COVID-?,. ¡Hemos terminado! Gracias por participar en esta tarea.
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"[6.68339E+15, 6.68339E+15, 6683389409361920, 6...","[4767148365053952, 5.49012E+15, 52135476029030...","[ES, ES, ES, ES, ES, ES]","[129468, 129468, 129523, 129523, 129523, 129468]",NaN,"[7.9137219596446675,-72.545011018832511, 7.913...",[http://storage.googleapis.com/prod-premise-an...,[[http://storage.googleapis.com/prod-premise-a...,"[Si, Si, Si, Si, Si, Si]","[Entre 3 y 5 años, Entre 3 y 5 años, No estoy ...","[No, No, No, No, No, No]",[http://storage.googleapis.com/prod-premise-an...,"[[Recolección de agua de lluvia, Una vivienda ...","[[7.9117929,-72.5463268, 7.9118691,-72.5463518...",[[http://storage.googleapis.com/prod-premise-a...,"[[], [], [], [], [], []]","[Si, Si, Si, Si, Si, Si]","[Algunas de las casas tienen puertas, Si, toda...",[Madera | Latón | Metal corrugado | Lad...,[http://storage.googleapis.com/prod-premise-an...,"[Si, No, No, Si, Si, No]","[Alfonso Gómez 2, nan, nan, Alfonso Gomez, Alf...",[Hay una mezcla de residentes colom

In [ ]:
# Print the original column names
for col in MasterDict.keys():
  print(col)

form_id
submission_id
locale
location_id
location_name
location_centroid
. Por favor tome una foto del asentamiento
. Por favor tome una foto del asentamiento desde un ángulo diferente
. ¿Existe un asentamiento en este punto?
. ¿Sabe desde hace cuanto comenzó el asentamiento?
. ¿Las calles están pavimentadas?
. Por favor tome una foto de una calle que muestre si está o no pavimentada
. Seleccione el aspecto de la comunidad que está reportando
. Díganos desde qué lugar esta reportando este elemento
. Por favor tome una foto del elemento que está reportando
. Por favor escriba en pocas palabras a qué se refiere con “otro” en la pregunta anterior.
. ¿El material de construcción de las viviendas parece ser improvisado?
. ¿Las casas tienen puertas?
. ¿De qué materiales están hechas las viviendas principalmente?
. Por favor tome una fotografía afuera de una vivienda o grupo de viviendas que nos permita entender mejor el tipo de material y cómo están construidas.
. ¿Sabe el nombre de este ase

In [ ]:
# Using the Official Dictionary (https://docs.google.com/document/d/1ndzWfIIWBdPW4VuWG18oPgbHN-3HJlB4y4DSGqgr9WM/edit?usp=sharing),
  #match each original column name with the shorter variable name,
  #creating a new sub-dictionary for each group of variables, e.g. var_variable = {corto: [largo,largo]}
  #and under each group-level sub-dictionary, a variable-level glossary for polichotomous variables, e.g. dict_var = {largo:corto}

var_etiqueta = {'Asentamiento':'location_name',
                'Posible nombre':'. Por favor escriba el nombre de este asentamiento',
                'Coordenadas':'location_centroid',
                'Departamento':'Departamento',
                'Municipio':'Municipio'}

#General
var_gen = {'Existe':'. ¿Existe un asentamiento en este punto?',
           'Composición': '. ¿Los residentes actuales del asentamiento son colombianos?',
           'Tiempo de Conformación':'. ¿Sabe desde hace cuanto comenzó el asentamiento?',
           'Urbana/Rural': '. ¿Cómo describiría el área en la que se encuentra el asentamiento?'}

dict_exist = {} # no necesita
dict_comp = {'Hay una mezcla de residentes colombianos y residentes que vienen de otro país':'Mezcla indeterminada',
              'Si, hay más colombianos que residentes de otro país':'Mayoría colombiana',
              'No, hay más de otro país que colombianos':'Minoría colombiana',
              'No estoy seguro':'NS/NR'}
dict_conf = {'Hace no más de un año':'Menos de 1 año',
             'Hace no más de dos años':'1-2 años',
             'Entre 3 y 5 años':'3-5 años',
             'No estoy seguro':'NS/NR'}
dict_urbrur = {} # no necesita


#Salud
var_salud = {'Precauciones Covid-19':'. ¿Qué medidas observa se están implementando para prevenir o mitigar el COVID-?',
             'Presencia de gente adulta mayor':'. ¿Dentro del asentamiento se percibe gente adulta mayor?',
             'Centro de salud internacional o nacional':'. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?',
             'Cercanía de un centro de salud':'. ¿Cuánto tiempo requiere para acudir a la institución o centro de salud más cercana al asentamiento?'}


dict_covid = {'Ninguna precaución':'',
             'Solo uso de tapabocas/guantes':'',
             'Solo distanciamiento social':'',
             'Ambas precauciones':''}
dict_mayor = {} # no necesita
dict_centro = {} # no necesita
dict_cercanía = {} # falta hacer


#SAN
var_SAN_elem = {'Tienda de comida':['Una tienda o establecimiento de venta de abarrotes dentro del asentamiento','Tiendas de comida, pequeños supermercados, o mercados'], 
                'Cultivo/jardin':['Cultivo, unidad agrícola o pecuaria','Un cultivo, o un jardín comunitario para auto-consumo o para la venta','Un lugar en donde se observa criado de ganado'],
                'Compostaje':['Espacios de compostaje de residuos de cocina']}

var_SAN_poli = {'Alimentos disponibles':['. ¿Qué alimentos y abarrotes venden dentro del establecimiento?', '. ¿Cuáles de los siguientes alimentos están disponibles dentro del establecimiento?'],
                'Tamaño de tienda de comida':['. Los establecimientos de venta de alimentos y abarrotes son en su mayoría:'],
                'Calidad de alimentos ofertados':['. La calidad de los alimentos ofertados es:'],
                'Comedor/cocina comunitaria':['. ¿El Asentamiento cuenta con algún espacio comunitario para la transformación adecuada de los productos alimenticios para consumo, como cocinas comunitarias o comedores?'],
                'Capacidad de cocinas comunitarias':['. ¿Cuál es la capacidad de dichas cocinas comunitarias o comedores?'],
                'Tipo de productos en cultivos':['. ¿Qué tipos de productos observa en los cultivos o en las unidades productivas identificadas?'],
                'N° de cultivos':['. ¿Cuántas cultivos, unidades agrícolas o pecuarias hay en este asentamiento?']}

dict_alim = {} # falta hacer
dict_tamaño = {} # falta hacer
dict_calidadalim = {} # falta hacer
dict_comed = {} # falta hacer
dict_capac = {} # falta hacer
dict_numerodecultivos = {} # falta hacer

#WASH

var_WASH = {'Acueducto':['Acueducto'], 
             'Agua estancada':['Agua estancada con criaderos de mosquitos o lugares donde pueden reproducirse'], 
             'Barril de agua':['Barril de agua en el exterior de una vivienda','Barriles de agua al frente de algunas viviendas'],
             'Bomba o pozo':['Bomba de funcionamiento de motor','Perforación o pozo con bomba de motor','Pozo abierto protegido'], 
             'Agua superficial':['Agua superficial (río o lago)'], 
             'Fuente de agua':['Llave de agua comunitaria','Llave de agua pública o comunal','Otra fuente pública de agua'],
             'Recolección de lluvia':['Recolección de agua de lluvia'],
             'Tuberías de agua':['Public_or_private_water_tubes_or_pipes', 'Tubería de agua pública o privada'] , 
             'Tuberías de agua negra':['Tubos de PVC que lleven aguas sucias hacia un sitio común', 'Tubos que llevan aguas negras hacia un sitio público'],
             'Vendedor de agua':['Vendedores de agua (carrotanques privados)'],
             'Instalación pública sanitaria':['Instalación pública sanitaria'],
             'Bote de basura':['Bote de basura visible desde el exterior de la vivienda']}
#Material

var_mat = {'Material':'. ¿De qué materiales están hechas las viviendas principalmente?'}

var_mat_elem = {'Plástico':['Plástico corrugado','Láminas de plástico'],
                'Metal':['Metal corrugado','Latón'],
                'Madera':['Madera'],
                'Ladrillo':['Ladrillo'],
                'Barro':['Barro']}

#Infraestructura

var_infr_elem = {'Alumbrado público':['Alumbrado público'],
                 'Andenes':['Andenes'],
                 'Salones comunales':['Salones comunales'],
                 'Señales de tránsito':['Señales de tránsito'],
                 'Nueva construcción':['Nueva construcción','Una vivienda o un desarrollo de viviendas'],
                 'Transporte público':['Parada de autobús'], # ,'Paradas de transporte público'], #because this variable is missing strangely from the elemento_matrix
                 'Parques':['Parques','Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)'],
                 'Centro educativo':['Escuela o centro educativo'],
                 'Elemento de comunicación':['Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)']}
           
var_infr_poli = {'Calidad de señal celular':['. ¿Cómo calificaría la calidad de la señal de celular en la zona?'],
                 'Calles pavimentadas':['. ¿Las calles están pavimentadas?'],
                 'Construcción improvisada':['. ¿El material de construcción de las viviendas parece ser improvisado?'],
                 'Puertas':['. ¿Las casas tienen puertas?'],
                 'Condición de las viviendas':['. En términos generales ¿cómo son las condiciones de las viviendas?'],
                 'N° de construcciones nuevas':['. ¿ Cuántas construcciones (viviendas) nuevas percibe que hay?']}
       
dict_calidcelu = {} # falta hacer
dict_calle = {} # no necesita
dict_improv = {} # falta hacer
dict_puert = {} # falta hacer
dict_condic = {} # falta hacer
dict_numerodeconstr = {} # falta hacer

### FindMode Function

In [ ]:
# Define a function that takes the list/array of surveyor responses as an input
  #, and if there are multiple modes returns 'Mezcla indeterminada'
  #, otherwise if there is a single mode it returns the mode as the new_val

def FindMode(Responses):
  responses = Responses
  
  uq_responses = pd.unique(responses)

  a = []

  if (type(responses) == np.ndarray):
    responses = responses.tolist()

  for uq_res in uq_responses:
    a.append(responses.count(uq_res))

  m = max(a)

  idx = [i for i, j in enumerate(a) if j == m]
  idx

  if len(idx)>1:
    new_val = ('Mezcla indeterminada')
  else:
    most_freq = uq_responses[idx[0]]
    new_val = (most_freq)

  return new_val

## Pre-process SuperMatrix

In [ ]:
#Prepare SuperMatrix for some specific variables that have different reliability methodologies

# Prepare for Precaucion Covid-19
for asen in SuperMatrix.index:

  values = str(SuperMatrix.loc[asen,'. ¿Qué medidas observa se están implementando para prevenir o mitigar el COVID-?'])

  if ('Distanciamiento Social' not in values) and (('guantes' not in values) or ('tapabocas' not in values)):
    new_val = [['Ninguna precuación']]
  elif ('Distanciamiento Social' in values) and (('guantes' not in values) or ('tapabocas' not in values)):
    new_val = [['Solo distanciamiento social']]
  elif (('guantes' in values) or ('tapabocas' in values)) and ('Distanciamiento Social' not in values):
    new_val = [['Solo uso de tapabocas/guantes']]
  elif (('guantes' in values) or ('tapabocas' in values)) and ('Distanciamiento Social' in values):
    new_val = [['Ambas precauciones']]
  else:
    new_val = 'Error: Condition not accounted for in If statements'
    print(asen +  ': Error: Condition not accounted for in If statements')
  
  SuperMatrix.loc[asen,'. ¿Qué medidas observa se están implementando para prevenir o mitigar el COVID-?'] = new_val

In [ ]:
# Prepare for Centro de salud internacional o nacional

for asen in SuperMatrix.index:
  old_val = SuperMatrix.loc[asen,'. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?']
  if type(old_val) != np.float:
    old_val = FindMode(old_val)
    if old_val == 'Ninguna de las anteriores':
      new_val = 'Ninguna'
    else:
      new_val = 'Si'
  elif (mt.isnan(old_val)):
    new_val = 'NS/NR'

  SuperMatrix.loc[asen,'. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?'] = new_val

## Create an Element Matrix

In [ ]:
# Create a matrix of elements

df3 = SuperMatrix[['Departamento','Municipio','. Seleccione el aspecto de la comunidad que está reportando']]

df3 = df3.rename(columns = {'. Seleccione el aspecto de la comunidad que está reportando' : 'Elemento'})

for asen in df3.index:
  lista = []
  for i in (df3.loc[asen,'Elemento']):
    for x in i:
      lista.append(x)
  
  lista_2 = str(lista).strip('[')
  lista_2 = lista_2.strip(']')
  df3.loc[asen,'Elemento'] = lista_2

df4 = df3.assign(Elemento = df3.Elemento.str.split('\', ')).explode('Elemento')

for i in range (0,len(df4)):
  df4.iloc[i,2] = df4.iloc[i,2].strip('\'')

dummies = pd.get_dummies(df4.Elemento)
df5 = pd.concat([df4,dummies],axis=1)

df5 = df5.reset_index(drop=False)

nueva_df5 = df5.groupby(['Asentamiento']).max()

#Subset the elements (make sure this includes all the elements)
nueva_df5 = nueva_df5[['Acueducto',
       'Agua estancada con criaderos de mosquitos o lugares donde pueden reproducirse',
       'Agua superficial (río o lago)', 'Alumbrado público', 'Andenes',
       'Barril de agua en el exterior de una vivienda',
       'Barriles de agua al frente de algunas viviendas',
       'Bomba de funcionamiento de motor',
       'Bote de basura visible desde el exterior de la vivienda',
       'Cultivo, unidad agrícola o pecuaria',
       'Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)',
       'Escuela o centro educativo',
       'Espacios de compostaje de residuos de cocina',
       'Instalación pública sanitaria', 'Llave de agua comunitaria',
       'Llave de agua pública o comunal', 'Nueva construcción',
       'Otra fuente pública de agua',
       'Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)',
       'Parada de autobús','Parques', 'Perforación o pozo con bomba de motor',
       'Pozo abierto protegido', 'Public_or_private_water_tubes_or_pipes',
       'Recolección de agua de lluvia', 'Salones comunales',
       'Señales de tránsito',
       'Tiendas de comida, pequeños supermercados, o mercados',
       'Tubería de agua pública o privada',
       'Tubos de PVC que lleven aguas sucias hacia un sitio común',
       'Tubos que llevan aguas negras hacia un sitio público',
       'Un cultivo, o un jardín comunitario para auto-consumo o para la venta',
       'Un lugar en donde se observa criado de ganado',
       'Una tienda o establecimiento de venta de abarrotes dentro del asentamiento',
       'Una vivienda o un desarrollo de viviendas',
       'Vendedores de agua (carrotanques privados)']]

#donde esta el elemento ['Paradas de transporte público'], sale en el MasterDict pero no aqui...

In [ ]:
elemento_matrix = nueva_df5.copy()

elemento_matrix.head()

,Acueducto,Agua estancada con criaderos de mosquitos o lugares donde pueden reproducirse,Agua superficial (río o lago),Alumbrado público,Andenes,Barril de agua en el exterior de una vivienda,Barriles de agua al frente de algunas viviendas,Bomba de funcionamiento de motor,Bote de basura visible desde el exterior de la vivienda,"Cultivo, unidad agrícola o pecuaria","Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)",Escuela o centro educativo,Espacios de compostaje de residuos de cocina,Instalación pública sanitaria,Llave de agua comunitaria,Llave de agua pública o comunal,Nueva construcción,Otra fuente pública de agua,"Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)",Parada de autobús,Parques,Perforación o pozo con bomba de motor,Pozo abierto protegido,Public_or_private_water_tubes_or_pipes,Recolección de agua de lluvia,Salones comunales,Señales de tránsito,"Tiendas de comida, pequeños supermercados, o mercados",Tubería de agua pública o privada,Tubos de PVC que lleven aguas sucias hacia un sitio común,Tubos que llevan aguas negras hacia un sitio público,"Un cultivo, o un jardín comunitario para auto-consumo o para la venta",Un lugar en donde se observa criado de ganado,Una tienda o establecimiento de venta de abarrotes dentro del asentamiento,Una vivienda o un desarrollo de viviendas,Vendedores de agua (carrotanques privados)
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,1,1,0,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
Arauca 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create and Fill Clean Matrix

In [ ]:
#Create a clean matrix with all the settlements and the first six columns
CleanMatrix = SuperMatrix[['Departamento','Municipio',	'N° de Validaciones',	'N° de Monitoreos',	'N° de Respuestas Validación',	'N° de Respuestas Monitoreo']]

### FillCleanMatrix 'Function'

In [ ]:
# Define a function to automatically fill the clean matrix with 
  #the mode value in the super matrix, 'Mezcla Indeterminada', or 'NS/NR'

def FillMatrix(VarDict):
  for key in VarDict.keys():
    
    #initialize the old and new column names to iterate through a set of variables
    col_new = key
    col_org = VarDict.get(key)

    #create a new column with the new short column name
    CleanMatrix[col_new] = empty_col

    for asen in SuperMatrix.index:
      responses = SuperMatrix.loc[asen,col_org]

      #Check whether the value in the SuperMatrix is a np array/list or a missing value/float,
        #and if it is an array, make it a list and run the FindMode funciton, returning the mode or 'Mezcla indeterminada'
        #but if it is a float, 
      if ((type(responses)==np.ndarray)):
        responses = responses.tolist()
        new_val = FindMode(responses)
      elif (type(responses)== list):
        new_val = FindMode(responses)
      elif type(responses)== float:
        if (mt.isnan(responses)==True):
          new_val = 'NS/NR'
      else:
        print('ERROR: Type ' +str(type(responses))+ ' not handled by this function I made!')

      #shorten the entries manually or by loading in variable specific dictionaries
      if new_val == 'No estoy seguro':
        new_val = 'NS/NR'

      if new_val == 'Hay una mezcla de residentes colombianos y residentes que vienen de otro país':
        new_val = 'Mezcla indeterminada'

      for key in dict_conf:
        if new_val == key:
          new_val = dict_conf.get(key)

      for key in dict_comp:
        if new_val == key:
          new_val = dict_comp.get(key)

      CleanMatrix.loc[asen,col_new] = new_val

  # Why do I get an error prompting me to use .loc when I already am using it?

### CheckDictionary Function

In [ ]:
def CheckElemDict(ElemDict):
  boooool = []

  for key in ElemDict:
    for col in ElemDict.get(key):
      b = col in elemento_matrix.columns.tolist()
      if b==False:
        print(str(b)+ ": " + col)
      boooool.append(b)
  print("Dictionary matches: " + str(all(boooool)))

### LongToShort Function

In [ ]:
def LongToShort(ElemDict,ElemMat):
  for key in ElemDict:
    lista_de_largos = ElemDict.get(key)
    for largo in lista_de_largos:
      
      corto = key
      ElemMat = ElemMat.rename(columns = {largo : corto})
  return ElemMat

### GroupElem Function

In [ ]:
#make a for loop that iterates through every key in the dictionary and every settlement and creates a cleaned element dataframe

def GroupElem(ElemDictDirty,ElemMat):
  d={}
  for key in ElemDictDirty.keys():
    d[key] = empty_col
  
  ElemDictClean = pd.DataFrame(index = ElemDictDirty.index,data=d)
  
  for key in ElemDictDirty.keys():
    for asen in ElemDictDirty.index:
      if type(ElemDictDirty.loc[asen,key].tolist()) == int: #If there is only one response, just grab the response
        val = ElemDictDirty.loc[asen,key]
      else: #But if there are more than one response (e.g. Barril de agua), assign the value a 1 if there is at least one 1 in the list
        if (ElemDictDirty.loc[asen,key].tolist().count(1) > 0):
          val = 1
        else:
          val = 0
      ElemDictClean.loc[asen,key] = val

  return ElemDictClean

### Variables de Etiqueta

In [ ]:
var_etiqueta

{'Asentamiento': 'location_name',
 'Coordenadas': 'location_centroid',
 'Departamento': 'Departamento',
 'Municipio': 'Municipio',
 'Posible nombre': '. Por favor escriba el nombre de este asentamiento'}

In [ ]:
# Coordenada

empty_col = (np.full(len(SuperMatrix), ''))

CleanMatrix['Coordenadas'] = empty_col

for i in range(0,len(SuperMatrix)):
  CleanMatrix.loc[CleanMatrix.index[i],'Coordenadas'] = SuperMatrix['location_centroid'][i][0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Variables generales

In [ ]:
var_gen

{'Composición': '. ¿Los residentes actuales del asentamiento son colombianos?',
 'Existe': '. ¿Existe un asentamiento en este punto?',
 'Tiempo de Conformación': '. ¿Sabe desde hace cuanto comenzó el asentamiento?',
 'Urbana/Rural': '. ¿Cómo describiría el área en la que se encuentra el asentamiento?'}

In [ ]:
FillMatrix(var_gen)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
CleanMatrix['Existe'].unique()
CleanMatrix['Tiempo de Conformación'].unique()
CleanMatrix['Composición'].unique()
CleanMatrix['Urbana/Rural'].unique()

array(['Urbana', 'NS/NR', 'Rural', 'Periurbana', 'Mezcla indeterminada'],
      dtype=object)

In [ ]:
CleanMatrix

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,Coordenadas,Existe,Composición,Tiempo de Conformación,Urbana/Rural
Asentamiento,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"7.9137219596446675,-72.545011018832511",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana
Arauca 1,Arauca,Arauca,1.0,0.0,33.0,NaN,"7.0774010352701158,-70.738436922992321",Si,Mezcla indeterminada,1-2 años,NS/NR
Arauca 6,Arauca,Arauca,1.0,0.0,23.0,NaN,"7.0626514753507346,-70.742873367335321",Si,Mezcla indeterminada,NS/NR,NS/NR
Arauca 7,Arauca,Arauca,1.0,0.0,37.0,NaN,"7.0801870927625581,-70.735573158824",Si,Mezcla indeterminada,1-2 años,NS/NR
Arauquita 11,Arauca,None,1.0,0.0,20.0,NaN,"7.0248856195254463,-71.4326067861188",Si,Mezcla indeterminada,3-5 años,NS/NR
...,...,...,...,...,...,...,...,...,...,...,...
Uribia-2,La Guajira,None,3.0,3.0,153.0,456.0,"11.734834066402694,-72.2672915671383",Si,Minoría colombiana,3-5 años,Periurbana
Uribia-3,La Guajira,None,3.0,3.0,129.0,488.0,"11.713913623825952,-72.2827792763549",Si,Minoría colombiana,3-5 años,Periurbana
Uribia-4,La Guajira,None,3.0,3.0,129.0,464.0,"11.718180248231935,-72.292290748378363",Si,Mayoría colombiana,3-5 años,Periurbana


### Variables de Salud

In [ ]:
# Call relevant dictionaries
var_salud

dict_covid #I definitely have to update this dictionary
#dict_mayor



{'Ambas precauciones': '',
 'Ninguna precaución': '',
 'Solo distanciamiento social': '',
 'Solo uso de tapabocas/guantes': ''}

In [ ]:
FillMatrix(var_salud) # Fix errors with string class type...

ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function I made!
ERROR: Type <class 'str'> not handled by this function 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# Clean up Centro de salud internacional o nacional

for asen in CleanMatrix.index:

  old_val = str(CleanMatrix.loc[asen,'Centro de salud internacional o nacional'])

  if old_val == 'Ninguna de las anteriores':
    new_val = 'Ninguna'
  elif old_val == 'NS/NR':
    new_val = old_val
  elif old_val == 'Mezcla indeterminada':
    new_val = old_val
  else:
    new_val = 'Si'

  CleanMatrix.loc[asen,'Centro de salud internacional o nacional'] = new_val

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# Manually set the Clean Matrix 'centro de salud' column to match the corresponding prepped Super Matrix column

CleanMatrix['Centro de salud internacional o nacional'] = SuperMatrix['. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#CleanMatrix['Precauciones Covid-19'].unique()
#CleanMatrix['Presencia de gente adulta mayor'].unique()
#CleanMatrix['Centro de salud internacional o nacional'].unique() 
#CleanMatrix['Cercanía de un centro de salud'].unique()
#CleanMatrix

### Variables de SAN

In [ ]:
#Display SAN dictionaries

print(var_SAN_elem)
print(var_SAN_poli)

{'Tienda de comida': ['Una tienda o establecimiento de venta de abarrotes dentro del asentamiento', 'Tiendas de comida, pequeños supermercados, o mercados'], 'Cultivo/jardin': ['Cultivo, unidad agrícola o pecuaria', 'Un cultivo, o un jardín comunitario para auto-consumo o para la venta', 'Un lugar en donde se observa criado de ganado'], 'Compostaje': ['Espacios de compostaje de residuos de cocina']}
{'Alimentos disponibles': ['. ¿Qué alimentos y abarrotes venden dentro del establecimiento?', '. ¿Cuáles de los siguientes alimentos están disponibles dentro del establecimiento?'], 'Tamaño de tienda de comida': ['. Los establecimientos de venta de alimentos y abarrotes son en su mayoría:'], 'Calidad de alimentos ofertados': ['. La calidad de los alimentos ofertados es:'], 'Comedor/cocina comunitaria': ['. ¿El Asentamiento cuenta con algún espacio comunitario para la transformación adecuada de los productos alimenticios para consumo, como cocinas comunitarias o comedores?'], 'Capacidad de c

In [ ]:
# Check if the dictionary matches the original element names
CheckElemDict(var_SAN_elem)

Dictionary matches: True


In [ ]:
#Shorten the names and prep for combining
elemento_matrix = LongToShort(var_SAN_elem,elemento_matrix)

elemento_matrix.head()

,Acueducto,Agua estancada con criaderos de mosquitos o lugares donde pueden reproducirse,Agua superficial (río o lago),Alumbrado público,Andenes,Barril de agua en el exterior de una vivienda,Barriles de agua al frente de algunas viviendas,Bomba de funcionamiento de motor,Bote de basura visible desde el exterior de la vivienda,Cultivo/jardin,"Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)",Escuela o centro educativo,Compostaje,Instalación pública sanitaria,Llave de agua comunitaria,Llave de agua pública o comunal,Nueva construcción,Otra fuente pública de agua,"Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)",Parada de autobús,Parques,Perforación o pozo con bomba de motor,Pozo abierto protegido,Public_or_private_water_tubes_or_pipes,Recolección de agua de lluvia,Salones comunales,Señales de tránsito,Tienda de comida,Tubería de agua pública o privada,Tubos de PVC que lleven aguas sucias hacia un sitio común,Tubos que llevan aguas negras hacia un sitio público,Cultivo/jardin,Cultivo/jardin,Tienda de comida,Una vivienda o un desarrollo de viviendas,Vendedores de agua (carrotanques privados)
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,1,1,0,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
Arauca 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Take the element matrix, subset the SAN elements and store in a separate matrix

SAN_matrix = elemento_matrix[list(var_SAN_elem.keys())]

SAN_matrix.head()

,Tienda de comida,Tienda de comida,Cultivo/jardin,Cultivo/jardin,Cultivo/jardin,Compostaje
Asentamiento,,,,,,
Alfonso Gomez,0,1,1,0,0,0
Arauca 1,1,0,0,0,0,0
Arauca 6,0,0,0,0,0,0
Arauca 7,1,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0


In [ ]:
SAN_matrix = GroupElem(SAN_matrix,elemento_matrix)

SAN_matrix

,Tienda de comida,Cultivo/jardin,Compostaje
Asentamiento,,,
Alfonso Gomez,1,1,0
Arauca 1,1,0,0
Arauca 6,0,0,0
Arauca 7,1,0,0
Arauquita 11,0,0,0
...,...,...,...
Uribia-2,0,1,0
Uribia-3,0,1,0
Uribia-4,0,1,0


In [ ]:
CleanMatrix = pd.concat([CleanMatrix,SAN_matrix],axis = 1)
CleanMatrix.head(3)

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,Coordenadas,Existe,Composición,Tiempo de Conformación,Urbana/Rural,Precauciones Covid-19,Presencia de gente adulta mayor,Centro de salud internacional o nacional,Cercanía de un centro de salud,Tienda de comida,Cultivo/jardin,Compostaje
Asentamiento,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"7.9137219596446675,-72.545011018832511",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,1,1,0
Arauca 1,Arauca,Arauca,1.0,0.0,33.0,NaN,"7.0774010352701158,-70.738436922992321",Si,Mezcla indeterminada,1-2 años,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,1,0,0
Arauca 6,Arauca,Arauca,1.0,0.0,23.0,NaN,"7.0626514753507346,-70.742873367335321",Si,Mezcla indeterminada,NS/NR,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,0,0,0


In [ ]:
# FillMatrix(var_SAN_poli) # how to find out what the error is here

### Variables de WASH

In [ ]:
#Display WASH dictionary
var_WASH

{'Acueducto': ['Acueducto'],
 'Agua estancada': ['Agua estancada con criaderos de mosquitos o lugares donde pueden reproducirse'],
 'Agua superficial': ['Agua superficial (río o lago)'],
 'Barril de agua': ['Barril de agua en el exterior de una vivienda',
  'Barriles de agua al frente de algunas viviendas'],
 'Bomba o pozo': ['Bomba de funcionamiento de motor',
  'Perforación o pozo con bomba de motor',
  'Pozo abierto protegido'],
 'Bote de basura': ['Bote de basura visible desde el exterior de la vivienda'],
 'Fuente de agua': ['Llave de agua comunitaria',
  'Llave de agua pública o comunal',
  'Otra fuente pública de agua'],
 'Instalación pública sanitaria': ['Instalación pública sanitaria'],
 'Recolección de lluvia': ['Recolección de agua de lluvia'],
 'Tuberías de agua': ['Public_or_private_water_tubes_or_pipes',
  'Tubería de agua pública o privada'],
 'Tuberías de agua negra': ['Tubos de PVC que lleven aguas sucias hacia un sitio común',
  'Tubos que llevan aguas negras hacia un

In [ ]:
# Check if the dictionary matches the original element names
CheckElemDict(var_WASH)

Dictionary matches: True


In [ ]:
#Shorten the names and prep for combining
elemento_matrix = LongToShort(var_WASH,elemento_matrix)

elemento_matrix.head()

,Acueducto,Agua estancada,Agua superficial,Alumbrado público,Andenes,Barril de agua,Barril de agua,Bomba o pozo,Bote de basura,Cultivo/jardin,"Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)",Escuela o centro educativo,Compostaje,Instalación pública sanitaria,Fuente de agua,Fuente de agua,Nueva construcción,Fuente de agua,"Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)",Parada de autobús,Parques,Bomba o pozo,Bomba o pozo,Tuberías de agua,Recolección de lluvia,Salones comunales,Señales de tránsito,Tienda de comida,Tuberías de agua,Tuberías de agua negra,Tuberías de agua negra,Cultivo/jardin,Cultivo/jardin,Tienda de comida,Una vivienda o un desarrollo de viviendas,Vendedor de agua
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,1,1,0,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
Arauca 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Take the elment matrix, subset the WASH elements and store in a separate matrix

wash_matrix = elemento_matrix[list(var_WASH.keys())]

wash_matrix.head()

,Acueducto,Agua estancada,Barril de agua,Barril de agua,Bomba o pozo,Bomba o pozo,Bomba o pozo,Agua superficial,Fuente de agua,Fuente de agua,Fuente de agua,Recolección de lluvia,Tuberías de agua,Tuberías de agua,Tuberías de agua negra,Tuberías de agua negra,Vendedor de agua,Instalación pública sanitaria,Bote de basura
Asentamiento,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,1,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1
Arauca 1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
wash_matrix = GroupElem(wash_matrix,elemento_matrix)

In [ ]:
# Concatenate WASH matrix to the clean matrix

CleanMatrix = pd.concat([CleanMatrix,wash_matrix],axis = 1)
CleanMatrix.head(3)

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,Coordenadas,Existe,Composición,Tiempo de Conformación,Urbana/Rural,Precauciones Covid-19,Presencia de gente adulta mayor,Centro de salud internacional o nacional,Cercanía de un centro de salud,Tienda de comida,Cultivo/jardin,Compostaje,Acueducto,Agua estancada,Barril de agua,Bomba o pozo,Agua superficial,Fuente de agua,Recolección de lluvia,Tuberías de agua,Tuberías de agua negra,Vendedor de agua,Instalación pública sanitaria,Bote de basura
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"7.9137219596446675,-72.545011018832511",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,1,1,0,1,1,1,0,0,1,1,0,0,0,1,1
Arauca 1,Arauca,Arauca,1.0,0.0,33.0,NaN,"7.0774010352701158,-70.738436922992321",Si,Mezcla indeterminada,1-2 años,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Arauca 6,Arauca,Arauca,1.0,0.0,23.0,NaN,"7.0626514753507346,-70.742873367335321",Si,Mezcla indeterminada,NS/NR,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Variables de Material

In [ ]:
## Identify unique materials
lol = []
for x in range(0,len(validES['. ¿De qué materiales están hechas las viviendas principalmente?'])):
  mat = validES['. ¿De qué materiales están hechas las viviendas principalmente?'][x]
  
  if type(mat) != np.float:
    mat = mat.split(' | ')
    for i in range(0,len(mat)):
      mat[i] = mat[i].strip(' ')
    lol.append(mat)

uq_lol = []
for bundle in lol:
  for item in bundle:
    if uq_lol.count(item) == 0:
      uq_lol.append(item)

uq_lol

['Madera',
 'Latón',
 'Metal corrugado',
 'Ladrillo',
 'Plástico corrugado',
 'Láminas de plástico',
 'Barro']

In [ ]:
# Do something similar to fill CleanMatrix material columns with 1s and 0s for each of the 5 main materials

### Variables de Infraestructura

In [ ]:
#Display Infraestructura dictionaries

print(var_infr_elem)
print(var_infr_poli)

{'Alumbrado público': ['Alumbrado público'], 'Andenes': ['Andenes'], 'Salones comunales': ['Salones comunales'], 'Señales de tránsito': ['Señales de tránsito'], 'Nueva construcción': ['Nueva construcción', 'Una vivienda o un desarrollo de viviendas'], 'Transporte público': ['Parada de autobús', 'Paradas de transporte público'], 'Parques': ['Parques', 'Otros espacios de recreación social (kioskos, canchas de fútbol, etc.)'], 'Centro educativo': ['Escuela o centro educativo'], 'Elemento de comunicación': ['Elemento de información o comunicación (antenas de televisión en las viviendas, antenas de celulares,puestos de venta de periódicos o revistas)']}
{'Calidad de señal celular': ['. ¿Cómo calificaría la calidad de la señal de celular en la zona?'], 'Calles pavimentadas': ['. ¿Las calles están pavimentadas?'], 'Construcción improvisada': ['. ¿El material de construcción de las viviendas parece ser improvisado?'], 'Puertas': ['. ¿Las casas tienen puertas?'], 'Condición de las viviendas': [

In [ ]:
# Check if the dictionary matches the original element names
CheckElemDict(var_infr_elem)

#Note: 'Paradas de transporte público' was being wierd again... so I removed it from the element dictionary 

Dictionary matches: True


In [ ]:
#Shorten the names and prep for combining
elemento_matrix = LongToShort(var_infr_elem,elemento_matrix)

elemento_matrix.head()

,Acueducto,Agua estancada,Agua superficial,Alumbrado público,Andenes,Barril de agua,Barril de agua,Bomba o pozo,Bote de basura,Cultivo/jardin,Elemento de comunicación,Centro educativo,Compostaje,Instalación pública sanitaria,Fuente de agua,Fuente de agua,Nueva construcción,Fuente de agua,Parques,Transporte público,Parques,Bomba o pozo,Bomba o pozo,Tuberías de agua,Recolección de lluvia,Salones comunales,Señales de tránsito,Tienda de comida,Tuberías de agua,Tuberías de agua negra,Tuberías de agua negra,Cultivo/jardin,Cultivo/jardin,Tienda de comida,Nueva construcción,Vendedor de agua
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,1,1,0,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
Arauca 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Take the element matrix, subset the Infraestructura elements and store in a separate matrix

Infr_matrix = elemento_matrix[list(var_infr_elem.keys())]

Infr_matrix.head()

,Alumbrado público,Andenes,Salones comunales,Señales de tránsito,Nueva construcción,Nueva construcción,Transporte público,Parques,Parques,Centro educativo,Elemento de comunicación
Asentamiento,,,,,,,,,,,
Alfonso Gomez,0,0,0,0,1,1,1,0,0,0,1
Arauca 1,0,0,1,0,0,0,0,0,0,0,0
Arauca 6,0,0,0,0,0,0,0,0,0,0,0
Arauca 7,1,0,1,0,0,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Infr_matrix = GroupElem(Infr_matrix,elemento_matrix)

Infr_matrix

,Alumbrado público,Andenes,Salones comunales,Señales de tránsito,Nueva construcción,Transporte público,Parques,Centro educativo,Elemento de comunicación
Asentamiento,,,,,,,,,
Alfonso Gomez,0,0,0,0,1,1,0,0,1
Arauca 1,0,0,1,0,0,0,0,0,0
Arauca 6,0,0,0,0,0,0,0,0,0
Arauca 7,1,0,1,0,0,0,0,0,0
Arauquita 11,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
Uribia-2,0,0,0,0,1,0,0,0,1
Uribia-3,0,0,0,0,1,0,0,0,0
Uribia-4,0,0,0,0,1,0,0,0,1


In [ ]:
CleanMatrix = pd.concat([CleanMatrix,Infr_matrix],axis = 1)
CleanMatrix.head(3)

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,Coordenadas,Existe,Composición,Tiempo de Conformación,Urbana/Rural,Precauciones Covid-19,Presencia de gente adulta mayor,Centro de salud internacional o nacional,Cercanía de un centro de salud,Tienda de comida,Cultivo/jardin,Compostaje,Acueducto,Agua estancada,Barril de agua,Bomba o pozo,Agua superficial,Fuente de agua,Recolección de lluvia,Tuberías de agua,Tuberías de agua negra,Vendedor de agua,Instalación pública sanitaria,Bote de basura,Alumbrado público,Andenes,Salones comunales,Señales de tránsito,Nueva construcción,Transporte público,Parques,Centro educativo,Elemento de comunicación
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"7.9137219596446675,-72.545011018832511",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,1,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1
Arauca 1,Arauca,Arauca,1.0,0.0,33.0,NaN,"7.0774010352701158,-70.738436922992321",Si,Mezcla indeterminada,1-2 años,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Arauca 6,Arauca,Arauca,1.0,0.0,23.0,NaN,"7.0626514753507346,-70.742873367335321",Si,Mezcla indeterminada,NS/NR,NS/NR,Ninguna precuación,NS/NR,NS/NR,NS/NR,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# FillMatrix(var_infr_poli) # how to find out what the error is here

## Clean Matrix Exploration

In [ ]:
# Display the unique values for each of the columns in the Clean Matrix

for col in CleanMatrix.columns:
  print(col + ": " + str(CleanMatrix.loc[:,col].unique()))

## Visualizations

In [ ]:
nds = CleanMatrix[CleanMatrix['Departamento']=='Norte de Santander']
nds

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,Coordenadas,Existe,Composición,Tiempo de Conformación,Urbana/Rural,Precauciones Covid-19,Presencia de gente adulta mayor,Centro de salud internacional o nacional,Cercanía de un centro de salud,Tienda de comida,Cultivo/jardin,Compostaje,Acueducto,Agua estancada,Barril de agua,Bomba o pozo,Agua superficial,Fuente de agua,Recolección de lluvia,Tuberías de agua,Tuberías de agua negra,Vendedor de agua,Instalación pública sanitaria,Bote de basura,Alumbrado público,Andenes,Salones comunales,Señales de tránsito,Nueva construcción,Transporte público,Parques,Centro educativo,Elemento de comunicación
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"7.9137219596446675,-72.545011018832511",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,1,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1
Cucuta 1,Norte de Santander,Cúcuta,4.0,3.0,100.0,180.0,"7.8830118242685119,-72.531722159446872",Si,Mezcla indeterminada,Mezcla indeterminada,Urbana,Ninguna precuación,Unos pocos,Ninguna,En transporte público o automóvil menos de 20 ...,1,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1
Cucuta 2,Norte de Santander,Cúcuta,3.0,3.0,75.0,145.0,"7.9434489006159277,-72.491868855423576",Si,Minoría colombiana,3-5 años,Urbana,Ninguna precuación,Unos pocos,Ninguna,En transporte público o automóvil menos de 20 ...,1,1,0,1,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1
Cucuta 3,Norte de Santander,Cúcuta,3.0,3.0,92.0,170.0,"7.9141576237404783,-72.555442429088686",Si,Mezcla indeterminada,NS/NR,Urbana,Ninguna precuación,En gran cantidad,Ninguna,En transporte público o automóvil más 30 minut...,1,1,0,1,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1
Cucuta 4,Norte de Santander,Cúcuta,5.0,4.0,129.0,196.0,"7.9029505285086348,-72.562526872673516",Si,Mezcla indeterminada,NS/NR,Rural,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,0,1,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1
Cucuta 5,Norte de Santander,Cúcuta,3.0,3.0,72.0,116.0,"7.9112375834756143,-72.557552008649651",Si,Mezcla indeterminada,3-5 años,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvilmás de una hora,0,1,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1
Cucuta 6,Norte de Santander,Cúcuta,3.0,3.0,97.0,172.0,"7.9105093986822261,-72.555416306098877",Si,Mezcla indeterminada,3-5 años,Urbana,Ninguna precuación,Unos pocos,Ninguna,En transporte público o automóvil más 30 minut...,1,1,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1
El Talento Sector La Flor,Norte de Santander,Cúcuta,10.0,9.0,246.0,376.0,"7.9132058620853156,-72.549929656256353",Si,Mezcla indeterminada,3-5 años,Urbana,Ambas precauciones,Unos pocos,Ninguna,En transporte público o automóvil más 30 minut...,1,1,0,1,1,1,0,0,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1
La Conquista,Norte de Santander,Cúcuta,3.0,3.0,78.0,166.0,"7.945533468321309,-72.526284910795127",Si,Mezcla indeterminada,3-5 años,Urbana,Solo distanciamiento social,En gran cantidad,Ninguna,Caminando o en bicicleta más de 30 minutos,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1


In [ ]:
#Save the Norte de Santander

nds.to_csv('NorteDeSantander.csv')

In [ ]:
nds2 = SuperMatrix[SuperMatrix['Departamento']=='Norte de Santander']
nds2

,Departamento,Municipio,N° de Validaciones,N° de Monitoreos,N° de Respuestas Validación,N° de Respuestas Monitoreo,form_id,submission_id,locale,location_id,location_name,location_centroid,. Por favor tome una foto del asentamiento,. Por favor tome una foto del asentamiento desde un ángulo diferente,. ¿Existe un asentamiento en este punto?,. ¿Sabe desde hace cuanto comenzó el asentamiento?,. ¿Las calles están pavimentadas?,. Por favor tome una foto de una calle que muestre si está o no pavimentada,. Seleccione el aspecto de la comunidad que está reportando,. Díganos desde qué lugar esta reportando este elemento,. Por favor tome una foto del elemento que está reportando,. Por favor escriba en pocas palabras a qué se refiere con “otro” en la pregunta anterior.,. ¿El material de construcción de las viviendas parece ser improvisado?,. ¿Las casas tienen puertas?,. ¿De qué materiales están hechas las viviendas principalmente?,. Por favor tome una fotografía afuera de una vivienda o grupo de viviendas que nos permita entender mejor el tipo de material y cómo están construidas.,. ¿Sabe el nombre de este asentamiento?,. Por favor escriba el nombre de este asentamiento,. ¿Los residentes actuales del asentamiento son colombianos?,. Díganos desde qué lugar está reportando ete elemento,. Por favor tome una fotografía del elemento que está reportando,. Los establecimientos de venta de alimentos y abarrotes son en su mayoría:,. ¿Qué alimentos y abarrotes venden dentro del establecimiento?,. ¿Cuáles de los siguientes alimentos están disponibles dentro del establecimiento?,. La calidad de los alimentos ofertados es:,. Por favor tome una fotografía de un anaquel con algunos productos dentro de la tienda.,". ¿Cuántas cultivos, unidades agrícolas o pecuarias hay en este asentamiento?",. ¿Qué tipos de productos observa en los cultivos o en las unidades productivas identificadas?,. En términos generales ¿cómo son las condiciones de las viviendas?,. ¿Qué tipos de instalaciones públicas sanitarias existen para los habitantes del asentamiento?,. ¿ Cuántas construcciones (viviendas) nuevas percibe que hay?,". ¿El Asentamiento cuenta con algún espacio comunitario para la transformación adecuada de los productos alimenticios para consumo, como cocinas comunitarias o comedores?",. ¿Cuál es la capacidad de dichas cocinas comunitarias o comedores?,. ¿Cuánto tiempo requiere para acudir a la institución o centro de salud más cercana al asentamiento?,. ¿Cómo calificaría la calidad de la señal de celular en la zona?,. ¿Hay alguna institución de atención en salud de carácter internacional o nacional en el municipio cercano al asentamiento?,. ¿Cómo describiría el área en la que se encuentra el asentamiento?,. ¿Dentro del asentamiento se percibe gente adulta mayor?,. ¿Qué medidas observa se están implementando para prevenir o mitigar el COVID-?,. ¡Hemos terminado! Gracias por participar en esta tarea.
Asentamiento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alfonso Gomez,Norte de Santander,Cúcuta,6.0,6.0,180.0,354.0,"[6.68339E+15, 6.68339E+15, 6683389409361920, 6...","[4767148365053952, 5.49012E+15, 52135476029030...","[ES, ES, ES, ES, ES, ES]","[129468, 129468, 129523, 129523, 129523, 129468]",NaN,"[7.9137219596446675,-72.545011018832511, 7.913...",[http://storage.googleapis.com/prod-premise-an...,[[http://storage.googleapis.com/prod-premise-a...,"[Si, Si, Si, Si, Si, Si]","[Entre 3 y 5 años, Entre 3 y 5 años, No estoy ...","[No, No, No, No, No, No]",[http://storage.googleapis.com/prod-premise-an...,"[[Recolección de agua de lluvia, Una vivienda ...","[[7.9117929,-72.5463268, 7.9118691,-72.5463518...",[[http://storage.googleapis.com/prod-premise-a...,"[[], [], [], [], [], []]","[Si, Si, Si, Si, Si, Si]","[Algunas de las casas tienen puertas, Si, toda...",[Madera | Latón | Metal corrugado | Lad...,[http://storage.googleapis.com/prod-premise-an...,"[Si, No, No, Si, Si, No]","[Alfonso Gómez 2, nan, nan, Alfonso Gomez, Alf...",[Hay una mezcla de residentes colom

In [ ]:
nds2.to_csv('NorteDeSantander2.csv')

### Estatus de Validación y Monitoreo de Asentamientos en el Norte de Santander

In [ ]:
asen = nds.index
valid = nds['N° de Validaciones']
monit = nds['N° de Monitoreos']

x = asen
y = valid
w = monit

fig,ax = plt.subplots()  # a figure with a single Axes
ax.set_title("Estatus de Validación y Monitoreo de Asentamientos en el Norte de Santander")
ax.set_xlabel('Asentamiento')  # Add an x-label to the axes.
ax.set_ylabel('N° de Encuestas')  # Add a y-label to the axes.
plt.xticks(rotation=90)
#plt.yticks(stats_matrix['N° de Validaciones'].unique())

ax.scatter(x,y,label='Validación',color = (0,0,0,.75),marker='$V$')
ax.bar(x,y,label='Validación',color = (0,0,0,.75),width=0.05)
ax.scatter(x,w,label='Monitoreo',color = (0,0,0,0.50),marker='$M$')
ax.bar(x,w,label='Monitoreo',color = (0,0,0,0.25),width=0.05)

x_coordinates = [asen[0],asen[len(asen)-1]]
y_coordinatesMax = [nds['N° de Validaciones'].max(),nds['N° de Validaciones'].max()]
y_coordinates3 = [3,3]
y_coordinates2 = [2,2]
y_coordinates1 = [1,1]
y_coordinates0 = [0,0]

plt.plot(x_coordinates,y_coordinates3,color='g')
plt.plot(x_coordinates,y_coordinates2,color='r')

#plt.rcParams["figure.figsize"] = [8, 5]

ax.fill_between(x_coordinates,y_coordinatesMax,y_coordinates3,color='g',alpha=.25) #
ax.fill_between(x_coordinates,y_coordinates2,y_coordinates0,color='r',alpha=.25)#

ax.legend()  # Add a legend.

plt.show()

### Pie Charts

In [ ]:
def PieChart(Col):
  col = Col
  lbls = nds[col].tolist()

  sizes = []
  for res in pd.unique(lbls):
    sizes.append(lbls.count(res)/len(lbls))

  labels = pd.unique(lbls)
  sizes = sizes 

  fig1, ax1 = plt.subplots()
  ax1.set_title(col + ": Asentamientos en el Norte de Santander")

  ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
          shadow=True, startangle=90) 
  ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

  plt.show()

In [ ]:
#General Pie Charts

for key in var_gen.keys():
  PieChart(key)

In [ ]:
for key in var_WASH.keys():
  PieChart(key)